In [ ]:
# from google.colab import drive
# drive.mount("/content/drive/")

In [ ]:
import os
import glob
import gdal
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt

quad_path =''

#input file code here
# file1 = r'/content/drive/MyDrive/ISRO/ch2_tmc_ndn_20200105T1902278938_d_oth_mad.tif'
# ras = gdal.Open(file1)

# geoTransform1 = ras.GetGeoTransform()
# minx = geoTransform1[0]
# maxy = geoTransform1[3]
# maxx = minx + geoTransform1[1] * ras.RasterXSize + ras.RasterYSize*geoTransform1[2]
# miny = maxy + geoTransform1[5] * ras.RasterYSize + ras.RasterXSize*geoTransform1[4]
# print(minx,maxx,miny,maxy)
# print(minx + geoTransform1[1] * ras.RasterXSize)
# print(maxy + geoTransform1[5] * ras.RasterYSize)
# print(ras.RasterYSize*geoTransform1[2])
# print(ras.RasterXSize*geoTransform1[4])


In [ ]:
def sort_with_time(directory):
  images = [tiff for tiff in sorted(glob.glob(os.path.join(directory+"*.tif")))]
  j = []
  for image in images:
    ras = gdal.Open(image)
    j.append(ras.GetMetadataItem("TIFFTAG_DATETIME"))

  j.sort()
  return j

In [ ]:
def maxmin(quad_path):
  images = [tiff for tiff in sorted(glob.glob(os.path.join(quad_path+"*.tif")))]
  absmaxx = 0
  absminx=0
  absminy = 0
  absmaxy=0
  for image in images:
    ras = gdal.Open(image)
    geoTransform1 = ras.GetGeoTransform()
    maxx = minx + geoTransform1[1] * ras.RasterXSize + ras.RasterYSize*geoTransform1[2]
    minx = geoTransform1[0] 
    miny = maxy + geoTransform1[5] * ras.RasterYSize + ras.RasterXSize*geoTransform1[4]
    maxy = geoTransform1[3]
    if minx<absminx:
      absminx=minx
    if maxx>absmaxx:
      absmaxx=maxx
    if miny<absminy:
      absminy=miny
    if maxy>absmaxy:
      absmaxy=maxy

  z = [absminx,absmaxy,absmaxx,absminy] #lon = x, lat = y
  return z

In [ ]:
#Euclidean distance
def xDistance(lat1, lat2, lon1, lon2):
     # return [abs(lon1-lon2), abs(lat1-lat2)]
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of moon in kilometers
    r = 1737.4
    # print(c,r)
    pixels = (c * r * 1000)/5 * 10 ** 14 * 1.3 
    # calculate the result
    return pixels

xDistance(85, 85, -180, 180)
xDistance(0, 0, -180, 180)
xDistance(30, 30, -180, 180)
# distance(90, -90, -60, -60)

In [ ]:
#make input quad_path sorted acc to time in ascending order
def stitch_quad(quad_path):
  z = maxmin(quad_path) # lon, lat, lon, lat
  #can we calculate x and y distance along any lat?
  counter = 0
  x_dist_lat = max(xDistance(z[1],z[1],z[0],z[2]), xDistance(z[3],z[3],z[0],z[2]))
  y_dist_lon = 5480 * abs(z[1]-z[3])/180 #5480 km
  x_res = x_dist_lat * 10 ** 3 /5  
  y_res = y_dist_lon * 10 ** 3 /5
  # if (abs(x_res - distance(z[3],z[3],z[0],z[2]))>1e-3):
  #   print('error')
  result = np.zeros(x_res,y_res)
  edge_detection = np.zeros(x_res,y_res, dtype=bool)
  sorted_with_time = sort_with_time(quad_path)
 #if no rotation
  for im_path in sorted_with_time:
    tif = gdal.Open(im_path)
    gt = tif.GetGeoTransform()
    if(gt[2]==0 and gt[4]==0):
      # x_len = gt.RasterXSize
      # y_len = gt.RasterYSize
      # print(x_len, y_len)
      myarray = np.array(tif.GetRasterBand(1).ReadAsArray())
      # x_new = xDistance(gt[3],gt[3],gt[0],z[0]) * 10**3 /5
      x_new = int(x_dist_lat/xDistance(gt[3],gt[3],gt[0],z[0])) * xDistance(gt[3],gt[3],gt[0],z[0]) * 10**3 /5
      y_new = y_res * abs(gt[3]-max(z[1],z[3]))/abs(z[1]-z[3])
      result[x_new : x_new+myarray.shape[0]+1][y_new : y_new+myarray.shape[1]+1] = myarray[myarray>0]*0.5 + 0.5*result[x_new : x_new+myarray.shape[0]+1][y_new : y_new+myarray.shape[1]+1][myarray>0]
      edge_detection[x_new : x_new+myarray.shape[0]+1][y_new : y_new+myarray.shape[1]+1][result[x_new : x_new+myarray.shape[0]+1][y_new : y_new+myarray.shape[1]+1]==0] = True
    else:
      counter+=1

  if(counter!=0):
    print(counter)
  return result, edge_detection

In [ ]:
quad_map, seams = stitch_quad(quad_path)
kernal = (1/25)*np.array([[1, 3, 1],
                    [3, 9, 3],
                    [1, 3, 1]]) # gaussian_kernel
quad_map[seams] = signal.convolve2d(quad_map, kernal, mode='same', boundary='symm', fillvalue=0)

f = plt.figure()
plt.imshow(quad_map)
plt.show()